In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
if 'google.colab' in sys.modules:
    %pip install -q skorch
    %git clone https://github.com/youyinnn/medical_imaging_imbalancing.git
    %cd /content/medical_imaging_imbalancing/src/isic
sys.path.insert(0, os.path.abspath('..'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from isic_resnet import ISICResNet18, ISICResNet50
import isic_skorch_trainer
from isic_dataset import ISIC_2018
from skorch.callbacks import LRScheduler

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# torch.manual_seed(42);

isic_train = ISIC_2018()
isic_val = ISIC_2018(split='val')

isic_train_128 = ISIC_2018(size=(128, 128))
isic_val_128 = ISIC_2018(split='val', size=(128, 128))

isic_train_448 = ISIC_2018(size=(448))
isic_val_448 = ISIC_2018(split='val', size=(448))

MEL: 1113 (11.11%), NV: 6705 (66.95%), BCC: 514 (5.13%), AKIEC: 327 (3.27%), BKL: 1099 (10.97%), DF: 115 (1.15%), VASC: 142 (1.42%)
MEL: 21 (10.88%), NV: 123 (63.73%), BCC: 15 (7.77%), AKIEC: 8 (4.15%), BKL: 22 (11.4%), DF: 1 (0.52%), VASC: 3 (1.55%)
MEL: 1113 (11.11%), NV: 6705 (66.95%), BCC: 514 (5.13%), AKIEC: 327 (3.27%), BKL: 1099 (10.97%), DF: 115 (1.15%), VASC: 142 (1.42%)
MEL: 21 (10.88%), NV: 123 (63.73%), BCC: 15 (7.77%), AKIEC: 8 (4.15%), BKL: 22 (11.4%), DF: 1 (0.52%), VASC: 3 (1.55%)
MEL: 1113 (11.11%), NV: 6705 (66.95%), BCC: 514 (5.13%), AKIEC: 327 (3.27%), BKL: 1099 (10.97%), DF: 115 (1.15%), VASC: 142 (1.42%)
MEL: 21 (10.88%), NV: 123 (63.73%), BCC: 15 (7.77%), AKIEC: 8 (4.15%), BKL: 22 (11.4%), DF: 1 (0.52%), VASC: 3 (1.55%)


In [45]:
epochs = 200

isic_skorch_trainer.net_fit(
    isic_skorch_trainer.net_def(
        ISICResNet18, 
        net_name = '18_256',
        classes=[torch.tensor, torch.tensor],    
        classifier_kwargs = dict(
            lr = 0.001,
            module__output_features = 7,
            train_split = isic_val,
            callbacks = [
                LRScheduler(policy='StepLR', step_size=7)]
            )
        ), 
isic_train, None, epochs)

  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      dur
-------  ------------  -----------  ----------  ------------  ----  -------
      1        0.6987       0.7824      0.5002        0.5686     +  62.7291
      2        0.4422       0.8187      0.6827        0.5642     +  57.4587
      3        0.3181       0.7876      0.5844        0.5620        57.7062
      4        0.2122       0.8446      0.6608        0.5298        57.8927
      5        0.1319       0.7979      0.6118        0.7766        56.7864
      6        0.0604       0.8135      0.6765        0.6551        56.6582
      7        0.0693       0.8083      0.7171        0.9616     +  56.3597
      8        0.0490       0.8342      0.6271        0.7999        54.1313
      9        0.0341       0.8187      0.6581        0.6533        53.7822
     10        0.0158       0.8238      0.7051        0.6773        56.8320
     11        0.0063       0.8290      0.7303        0.6953     +  60.8137
     12     

In [8]:
isic_skorch_trainer.net_fit(
    isic_skorch_trainer.net_def(
        ISICResNet18, 
        net_name = '18_448',
        classes=[torch.tensor, torch.tensor],    
        classifier_kwargs = dict(
            lr = 0.001,
            module__output_features = 7,
            train_split = isic_val_448,)
        ), 
isic_train_448, None, 0)

18_448_
